In [7]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [18]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import exposure
from PIL import ImageFilter

#### Function for histogram equalization

In [2]:
# Function to perform histogram equalization on a single channel
def histogram_equalization_channel(channel):
    # Convert the channel to a numpy array
    channel_array = np.array(channel)
    
    # Calculate the histogram
    hist, bins = np.histogram(channel_array.flatten(), 256, [0, 256])

    # Calculate cumulative distribution function (CDF)
    cdf = hist.cumsum()
    cdf_normalized = cdf * float(hist.max()) / cdf.max()  # Normalize CDF

    # Mask out all the zeros in the CDF
    cdf_m = np.ma.masked_equal(cdf, 0)

    # Perform histogram equalization by scaling the CDF
    cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())  # Normalize to [0,255]
    cdf = np.ma.filled(cdf_m, 0).astype('uint8')

    # Map the original channel values to the equalized values
    equalized_channel = cdf[channel_array]
    return Image.fromarray(equalized_channel)

In [3]:
def histogram_equalization_rgb(image):
    # Split the image into R, G, and B channels
    r, g, b = Image.Image.split(image)

    # Equalize each channel
    r_eq = histogram_equalization_channel(r)
    g_eq = histogram_equalization_channel(g)
    b_eq = histogram_equalization_channel(b)

    # Merge the equalized channels back together
    equalized_image = Image.merge('RGB', (r_eq, g_eq, b_eq))
    return equalized_image

#### Function for CLAHE

In [14]:
def clahe_im(image_dir, output_path):
    '''
    take in an image directory, and generate its clane version
    and save it to the output path.
    '''
    # Load an RGB image using PIL
    for image_name in os.listdir(image_dir):
        image = Image.open(os.path.join(image_dir,image_name))
    
        # Convert the image to a numpy array (shape: HxWxC)
        image_np = np.array(image)
        
        # Separate the RGB channels
        r, g, b = image_np[:,:,0], image_np[:,:,1], image_np[:,:,2]
        
        # Apply CLAHE on each channel individually
        r_clahe = exposure.equalize_adapthist(r, clip_limit=0.03, kernel_size=(8, 8))
        g_clahe = exposure.equalize_adapthist(g, clip_limit=0.03, kernel_size=(8, 8))
        b_clahe = exposure.equalize_adapthist(b, clip_limit=0.03, kernel_size=(8, 8))
        
        # Merge the CLAHE-processed channels back into a single image
        clahe_image_np = np.stack([r_clahe, g_clahe, b_clahe], axis=-1)
        
        # Convert the processed image back to a PIL image (scaled back to 0-255 range)
        clahe_image = Image.fromarray((clahe_image_np * 255).astype(np.uint8))
        
        # Save the output image
        clahe_image.save(os.path.join(output_path,image_name))

#### Creating HE pictures based on underexposed dataset

In [5]:
%%time

image_dir = 'C:/Users/leiyi/OneDrive/Desktop/data_sep/underexposed'
image_dir_sav = 'C:/Users/leiyi/OneDrive/Desktop/data_sep_new/underexposed/he'
for filename in os.listdir(image_dir):
    image = Image.open(os.path.join(image_dir,filename))
    equalized_image = histogram_equalization_rgb(image)
    equalized_image.save(os.path.join(image_dir_sav,filename))

CPU times: total: 55.9 s
Wall time: 1min 41s


#### Creating CLAHE pictures based on underexposed dataset

In [15]:
%%time
output_path = 'C:/Users/leiyi/OneDrive/Desktop/data_sep_new/underexposed/clahe'
clahe_im(image_dir,output_path)

CPU times: total: 11min 48s
Wall time: 41min 53s


#### Creating HE pictures based on verydark dataset

In [41]:
%%time

image_dir = 'C:/Users/leiyi/OneDrive/Desktop/data_sep/very dark'
image_dir_save = 'C:/Users/leiyi/OneDrive/Desktop/data_sep_new/very dark/he'
for filename in os.listdir(image_dir):
    image = Image.open(os.path.join(image_dir,filename))
    equalized_image = histogram_equalization_rgb(image)
    equalized_image.save(os.path.join(image_dir_save,filename))

CPU times: total: 46.1 s
Wall time: 2min 2s


#### Creating CLAHE pictures for very dark images

In [29]:
%%time
output_path = 'C:/Users/leiyi/OneDrive/Desktop/data_sep_new/very dark/clahe'
clahe_im(image_dir,output_path)

CPU times: total: 25min 31s
Wall time: 57min 21s


#### Creating HE pictures for less-saturated images

In [40]:
%%time

image_dir = 'C:/Users/leiyi/OneDrive/Desktop/data_sep/less-saturated'
image_dir_save = 'C:/Users/leiyi/OneDrive/Desktop/data_sep_new/less-saturated/he'
for filename in os.listdir(image_dir):
    image = Image.open(os.path.join(image_dir,filename))
    equalized_image = histogram_equalization_rgb(image)
    equalized_image.save(os.path.join(image_dir_save,filename))

CPU times: total: 51.8 s
Wall time: 2min


#### Creating CLAHE pictures for less-saturated images

In [32]:
%%time
output_path = 'C:/Users/leiyi/OneDrive/Desktop/data_sep_new/less-saturated/clahe'
clahe_im(image_dir,output_path)

CPU times: total: 17min 13s
Wall time: 55min 27s
